In [1]:
!pip install pandas numpy fastparquet


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd

# Read roads csv file

In [3]:
# detailed_trajectory = pd.read_pickle("./data_with_trajectory_20s/courier_detailed_trajectory_20s.pkl.xz",compression="xz")
# road_df=pd.read_csv("../LaDe/road-network/roads.csv",sep="\t")

road_df=pd.read_csv("../processed/roads_translated_motherfile.csv")
print(road_df.head(3))

print(" ----------------------------------------------------------------------------- ")

print(" ----------------------------------------------------------------------------- ")

delivery = pd.read_csv("../LaDe/delivery/delivery_hz.csv")
print(delivery.head(3))

print(" ----------------------------------------------------------------------------- ")
print(" ----------------------------------------------------------------------------- ")







    osm_id  code     fclass  name  ref oneway  maxspeed  layer bridge tunnel  \
0  4296533  5141    service   NaN  NaN      B         0      0      F      F   
1  4296592  5115   tertiary   朝晖路  NaN      B         0      0      F      F   
2  4298707  5114  secondary  体育场路  NaN      F         0      0      F      F   

       city                                           geometry city_chinese  
0  Hangzhou  LINESTRING (13373499.140421594 3538916.2947984...          杭州市  
1  Hangzhou  LINESTRING (13376291.100042382 3539415.0794264...          杭州市  
2  Hangzhou  LINESTRING (13375063.92510783 3538697.18123532...          杭州市  
 ----------------------------------------------------------------------------- 
 ----------------------------------------------------------------------------- 
   order_id  region_id      city  courier_id        lng       lat  aoi_id  \
0    583722          0  Hangzhou         175  120.17895  30.26401     749   
1   2819059          0  Hangzhou         175  120.178

# reading all postmanid into a folder parquet from a single pcikle file 

In [4]:
trajectory20s = pd.read_pickle("../LaDe/data_with_trajectory_20s/courier_detailed_trajectory_20s.pkl.xz",compression="xz")

trajectory20s.head(3)

,ds,postman_id,gps_time,lat,lng
0,321,106f5ac22cfd1574b196d16fed62f90d,03-21 07:31:58,3.953700e+06,3.053400e+06
1,321,106f5ac22cfd1574b196d16fed62f90d,03-21 07:32:18,3.953700e+06,3.053398e+06
2,321,106f5ac22cfd1574b196d16fed62f90d,03-21 07:32:41,3.953700e+06,3.053398e+06


In [5]:
import lzma, pickle, os
from pathlib import Path
import pandas as pd
import tempfile
PARQUET_COMPRESSION = 'snappy' 

DF_VAR = 'trajectory20s'                      # replace if your DataFrame variable name differs
OUT_DIR = Path('../processed/per_postman_pickles')
OUT_DIR.mkdir(exist_ok=True)
            

In [6]:
# assume `df` is already in memory (loaded from pickle)
# ensure postman_id column exists and is string
df = globals().get(DF_VAR)
if df is None:
    raise SystemExit("DataFrame variable 'df' not found in globals(). Please assign your DataFrame to variable `df`.")

if 'postman_id' not in df.columns:
    raise SystemExit("Column 'postman_id' not found in DataFrame.")

df['postman_id'] = df['postman_id'].astype(str)

In [7]:
# find top 100 postmen by record count
top50 = df['postman_id'].value_counts().nlargest(50).index.tolist()


# write per-postman parquet files
for pid in top50:
    sub = df[df['postman_id'] == pid].copy()
    out_path = OUT_DIR / f'postman_{pid}.parquet'
    sub.to_parquet(out_path, index=False, compression=PARQUET_COMPRESSION)

print(f"Saved {len(top50)} parquet files to: {OUT_DIR.resolve()}")

Saved 50 parquet files to: /home/aymuos/Documents/Github/starship/processed/per_postman_pickles


In [ ]:
courier_1 = pd.read_parquet("../processed/per_postman_pickles/postman_0c032cafea40c06fa6337701caf35607.parquet")
courier_1.shape

(80082, 5)

Key steps to perform
- Normalize timestamps and sort by time per postman.
- Drop exact duplicates (same timestamp+coords).
- Remove outliers (bad coords, nan lat/lng).
- Compute step metrics: dt (seconds), haversine distance (m), speed = distance / dt.
- Remove zero/near-zero-movement duplicates: consecutive points with distance < d_eps (e.g., 2–5 m) and dt small—keep only the first in run.
- Collapse long stops: detect stationary segments where speed < v_thresh (e.g., 0.5 m/s) for duration >= t_stop (e.g., 10+ minutes); keep only one representative point per stop (start point or centroid + aggregated stats).
- Remove implausible jumps: speed > v_max (e.g., 40 m/s ~ 144 km/h) — drop or interpolate.
- Optional smoothing: rolling median on lat/lng to reduce GPS jitter before stop detection.
- Save cleaned per-postman parquet.
- Recommended default thresholds (adjust to your city/device)

- d_eps = 5 meters
- v_thresh = 0.5 m/s
- t_stop = 600 seconds (10 min)
- v_max = 40 m/s
- min_dt = 5 seconds (ignore dt=0)
- Concise runnable example (apply per parquet file)

## idea -> 

Aggregation rules (LaDe repo -inspired)
For stationary segments longer than t_stop seconds, collapse into one aggregated "stop" with centroid, duration, total_dist, mean_speed, n_points.
For movement segments and short stops, keep endpoints and a summary segment row.
Produce cleaned trajectory with representative points (stop centroids or segment endpoints).
Why: long stops (courier waiting/serving) contribute little to route geometry but matter as tasks; collapsing reduces data volume while preserving task-level info for modeling. Keeping endpoints for moves preserves route shape with much fewer points.



- Historical averages
What: optional merge of a history DataFrame (previous segment records) to compute per-cluster historical average duration and visit counts, attaching these to new segments.
Why: LaDe-style time prediction benefits from historical statistics per stop (how long stops typically last, how often visited) to predict future durations.

In [17]:
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np
from datetime import timedelta

# Haversine (meters)
def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = np.radians(lat1); phi2 = np.radians(lat2)
    dphi = phi2 - phi1
    dl = np.radians(lon2 - lon1)
    a = np.sin(dphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dl/2.0)**2
    return R * 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

In [18]:
# hyper parameters 

MIN_MOVEMENT_DISTANCE = 5.0         # meters to consider movement
MAX_SPEED = 40.0,        # max plausible speed m/s
SECS_AGGREGATED = 600   # 10 mins aggregate 

DBSCAN_CLUSTER = 30

In [19]:
# clean dataframe 

def process_postman(df,
                    postman_id,
                    d_eps=MIN_MOVEMENT_DISTANCE , 
                    v_max= MAX_SPEED,
                    t_stop=SECS_AGGREGATED ,
                    min_dt=1,
                    stop_cluster_eps=DBSCAN_CLUSTER,   # meters for DBSCAN stop clustering
                    stop_cluster_min_samples=1,
                    history_df=None):      # optional DataFrame of past segments for historical averages
    """
    Input:
      df: DataFrame for a single postman (may contain multiple days). Must have columns lat,lng and timestamp or ds+gps_time.
      postman_id: identifier (string/int)
      history_df: optional aggregated segments from previous runs to compute historical averages (same schema as segments_df)

    Returns:
      cleaned_df: reduced trajectory (representative points)
      segments_df: aggregated segments with LaDe-style features
      stop_clusters: DataFrame of identified stop clusters (centroid, cluster_id, count, avg_duration)
    """

    # --- normalize timestamps ---
    if 'timestamp' not in df.columns:
        
        
        df['timestamp'] = pd.to_datetime(df.get('gps_time', None), errors='coerce')
    df = df.dropna(subset=['lat','lng','timestamp']).copy()


    df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
    df['lng'] = pd.to_numeric(df['lng'], errors='coerce')


    df = df.dropna(subset=['lat','lng','timestamp'])

    df = df.sort_values('timestamp').reset_index(drop=True)

    if df.empty:
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    # --- compute step metrics ---
    df['lat_p'] = df['lat'].shift(1)  # returns a new DataFrame (or Series) where each value is shifted down by 1 row:
    df['lng_p'] = df['lng'].shift(1)
    df['t_p'] = df['timestamp'].shift(1)


    df['dt'] = (df['timestamp'] - df['t_p']).dt.total_seconds().fillna(0)
    df['dt_fixed'] = df['dt'].replace(0, min_dt)


    # haversine between previous and cect 
    df['dist'] = haversine_m(df['lat_p'].fillna(df['lat']), df['lng_p'].fillna(df['lng']), df['lat'], df['lng'])


    df['speed'] = df['dist'] / df['dt_fixed']

    
    # filter implausible jumps
    df = df[(df['speed'] <= v_max) | (df['dt'] == 0)].copy()
    df = df.reset_index(drop=True)

    # --- mark move vs stationary and segment ids ---
    df['is_move'] = df['dist'] >= d_eps
    df['seg_change'] = (df['is_move'] != df['is_move'].shift(1)).fillna(True)
    df['seg_id'] = df['seg_change'].cumsum()

    # --- aggregate segments into LaDe-style rows ---
    seg_rows = []
    cleaned_rows = []
    stop_centroids = []
    for seg_id, g in df.groupby('seg_id', sort=True):
        seg_is_move = bool(g['is_move'].iloc[0])
        start_ts = g['timestamp'].iloc[0]
        end_ts = g['timestamp'].iloc[-1]
        duration = (end_ts - start_ts).total_seconds()
        n_points = len(g)
        total_dist = g['dist'].sum()
        mean_speed = total_dist / max(duration, 1.0) if duration > 0 else 0.0
        weights = g['dt'].fillna(0).values
        if weights.sum() > 0:
            centroid_lat = (g['lat'] * weights).sum() / weights.sum()
            centroid_lng = (g['lng'] * weights).sum() / weights.sum()
        else:
            centroid_lat = g['lat'].mean()
            centroid_lng = g['lng'].mean()

        if (not seg_is_move) and duration >= t_stop:
            # aggregated stop segment
            seg_rows.append({
                'postman_id': postman_id,
                'seg_id': seg_id,
                'type': 'stop',
                'start_ts': start_ts,
                'end_ts': end_ts,
                'duration_s': duration,
                'n_points': n_points,
                'centroid_lat': centroid_lat,
                'centroid_lng': centroid_lng,
                'total_dist_m': total_dist,
                'mean_speed_m_s': mean_speed
            })
            cleaned_rows.append({
                'postman_id': postman_id,
                'timestamp': start_ts,
                'lat': centroid_lat,
                'lng': centroid_lng,
                'note': 'stop_agg',
                'seg_id': seg_id
            })
            stop_centroids.append((centroid_lat, centroid_lng, duration))
        else:
            # movement or short stop: keep endpoints
            first = g.iloc[0]
            last = g.iloc[-1]
            cleaned_rows.append({
                'postman_id': postman_id,
                'timestamp': first['timestamp'],
                'lat': first['lat'],
                'lng': first['lng'],
                'note': 'seg_start',
                'seg_id': seg_id
            })
            if n_points > 1:
                cleaned_rows.append({
                    'postman_id': postman_id,
                    'timestamp': last['timestamp'],
                    'lat': last['lat'],
                    'lng': last['lng'],
                    'note': 'seg_end',
                    'seg_id': seg_id
                })
            seg_rows.append({
                'postman_id': postman_id,
                'seg_id': seg_id,
                'type': 'move' if seg_is_move else 'short_stop',
                'start_ts': start_ts,
                'end_ts': end_ts,
                'duration_s': duration,
                'n_points': n_points,
                'centroid_lat': centroid_lat,
                'centroid_lng': centroid_lng,
                'total_dist_m': total_dist,
                'mean_speed_m_s': mean_speed
            })

    cleaned_df = pd.DataFrame(cleaned_rows).sort_values('timestamp').reset_index(drop=True)
    segments_df = pd.DataFrame(seg_rows).sort_values('start_ts').reset_index(drop=True)

    # --- identify stop clusters (DBSCAN) from stop centroids and also from segments' centroids of type 'stop' ---
    stop_rows = segments_df[segments_df['type']=='stop']
    stop_clusters = pd.DataFrame()
    if not stop_rows.empty:
        coords = stop_rows[['centroid_lat','centroid_lng']].to_numpy()
        # convert meters eps to radians approx? DBSCAN with haversine requires coords in radians and metric='haversine'
        # simpler: convert eps meters -> degrees approx (~111111 m per degree lat). Use eps_deg = eps_m / 111111
        eps_deg = stop_cluster_eps / 111111.0
        clustering = DBSCAN(eps=eps_deg, min_samples=stop_cluster_min_samples, metric='haversine' if False else 'euclidean')
        # DBSCAN on lat/lng in degrees with euclidean approximates small eps well
        labels = clustering.fit_predict(coords)
        stop_rows = stop_rows.assign(cluster_id=labels)
        # compute cluster summaries
        clusters = stop_rows.groupby('cluster_id').agg(
            cluster_count=('seg_id','count'),
            avg_duration_s=('duration_s','mean'),
            centroid_lat=('centroid_lat','mean'),
            centroid_lng=('centroid_lng','mean')
        ).reset_index()
        stop_clusters = clusters
        # attach cluster id to segments_df
        segments_df = segments_df.merge(stop_rows[['seg_id','cluster_id']], on='seg_id', how='left')

    # --- historical averages (from history_df) ---
    # compute per-cluster historical avg duration and counts if history provided
    if history_df is not None and not history_df.empty:
        # expect history_df has cluster_id, duration_s, centroid coords
        hist = history_df.copy()
        # if history has cluster coordinates, match by nearest cluster centroid (use simple join by rounding coords)
        if 'cluster_id' in hist.columns:
            # compute per-cluster aggregates
            hist_agg = hist.groupby('cluster_id').agg(
                hist_avg_duration_s=('duration_s','mean'),
                hist_visit_count=('seg_id','count')
            ).reset_index()
            stop_clusters = stop_clusters.merge(hist_agg, on='cluster_id', how='left')
            # attach historical estimates to segments_df via cluster_id
            segments_df = segments_df.merge(hist_agg, on='cluster_id', how='left')
        else:
            # fallback: compute global historical averages
            gavg = hist['duration_s'].mean()
            segments_df['hist_avg_duration_s'] = gavg
    else:
        # set NaNs
        segments_df['hist_avg_duration_s'] = np.nan

    # --- additional LaDe-style features per segment ---
    # time-based features
    segments_df['start_hour'] = pd.to_datetime(segments_df['start_ts']).dt.hour
    segments_df['start_dow'] = pd.to_datetime(segments_df['start_ts']).dt.dayofweek
    segments_df['is_night'] = segments_df['start_hour'].isin(list(range(0,7)))  # example
    # relative features
    segments_df['dist_per_point'] = segments_df['total_dist_m'] / segments_df['n_points'].replace(0,1)

    # return cleaned trajectory, segments, stop clusters
    return cleaned_df, segments_df, stop_clusters


In [23]:
df = pd.read_parquet("../processed/per_postman_pickles/postman_0c032cafea40c06fa6337701caf35607.parquet")  # raw loaded
cleaned, segments, clusters = process_postman(df, postman_id='321', history_df=None)


cleaned.head(4)
# # save outputs
# cleaned.to_parquet('per_postman_cleaned/postman_321.parquet', index=False)
# segments.to_parquet('per_postman_segments/postman_321_segments.parquet', index=False)
# clusters.to_parquet('per_postman_clusters/postman_321_clusters.parquet', index=False)

/tmp/ipykernel_98150/3745643329.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df.get('gps_time', None), errors='coerce')


""


In [12]:

# traj_df["gps_time"]=pd.to_datetime(traj_df["gps_time"])

# traj_gsort_df = (
#     traj_df.groupby("postman_id",group_keys=False)
#     .apply(lambda g:g.sort_values("gps_time"))
#     .reset_index(drop=True)
# )



# traj_basegroup_df = (
#     traj_df.groupby(["ds","postman_id"])
#     .size()
#     .reset_index(name="count_coordinates")
# )



In [13]:
road_df.columns

# osm_id = openstreetmap_id 
# 'code' = osm internal code schema 
# 'fclass', =  functional category of road -  motorway, primary, secondary, residential
#  'name', = official name availble in osm
#  'ref'   = Reference code (like highway numbers, e.g., “G60” for a national expressway). ex. S102 
#  'oneway', = Indicates if the road is one-way (yes/no). Critical for routing algorithms.
#  'maxspeed'= Maximum legal speed limit on the road segment (km/h).
#  'layer' = Vertical layer of the road (used when roads overlap, e.g., flyovers vs. tunnels) --> 
#  'bridge'= Boolean flag (yes/no) indicating if the segment is a bridge.
#  'tunnel'= Boolean flag (yes/no) indicating if the segment is a tunnel.
#  'city', = The city to which the road belongs (Shanghai, Hangzhou, Chongqing, Jilin, Yantai).
# 'geometry' = The polyline coordinates (latitude/longitude pairs) describing the exact shape of the road segment.

Index(['osm_id', 'code', 'fclass', 'name', 'ref', 'oneway', 'maxspeed',
       'layer', 'bridge', 'tunnel', 'city', 'geometry', 'city_chinese'],
      dtype='object')

# Messing with the trajectory data 

In [14]:
trajectory20s['ds'] = pd.to_datetime('2021-'+trajectory20s['ds'], format='%Y-%m-%d', errors='coerce')

trajectory20s.head(3)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U5'), dtype('int64')) -> None

In [ ]:
# traj_gsort_df["seq"]=(
#     traj_gsort_df.groupby("postman_id")[["lat","lng"]]
#     .apply(lambda g: (g != g.shift())).any(axis=1).cumsum()
#     .reset_index(level=0, drop=True)
# )


In [ ]:
# traj_spacegroup_df = (
#     traj_gsort_df.groupby(["postman_id","seq"])
#     .agg(
#         lat = ("lat","first"),
#         lng = ("lng","first"),
#         start_time = ("gps_time","min"),
#         end_time = ("gps_time","max"),
#         n_pings = ("gps_time","size")
#     )
#     .reset_index()
# )

In [ ]:
# traj_spacegroup_df.to_pickle("trajectory_spacegroup.pkl")